In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Define constants and paths
IMG_SIZE = 244
device = torch.device("cuda")
image_folder = 'D:/DATA/augmented_images'
val_df = pd.read_csv('df_test.csv')
BATCH_SIZE = 16

# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def load_image(self, img_path):
        img = Image.open(img_path).convert('RGB')
        img = img.resize((IMG_SIZE, IMG_SIZE))
        return img

    def __getitem__(self, idx):
        fname = self.df.loc[idx, 'filename']
        label = self.df.loc[idx, 'label']
        img_path = os.path.join(self.img_dir, fname)
        img = self.load_image(img_path)
        if self.transform:
            img = self.transform(img)  # Convert PIL to Tensor
        return img, label

# Instantiate EfficientNet B0 model
model = EfficientNet.from_pretrained('efficientnet-b0')
model.to(device)

# Load augmented checkpoint
checkpoint_aug = torch.load('FineTunedmodel_checkpoint_1_5.pth')
model.load_state_dict(checkpoint_aug['model_state_dict'])

# Freeze early layers
for param in list(model.parameters())[:5]:
    param.requires_grad = False

# Small LR for fine-tuning
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# DataLoader for validation set using your custom dataset
val_transform = transforms.Compose([transforms.ToTensor()])
val_ds = ImageDataset(val_df, 'D:/DATA/augmented_images', transform=val_transform)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=True)

# Set up early stopping
early_stopping_counter = 0
patience = 3  # Set your patience value
best_val_loss = float('inf')
criterion = nn.CrossEntropyLoss()
PATH = 'C:/Users/Sri Ram/Untitled Folder 8/FineTunedmodel_checkpoint_1_5.pth'

# Unfreeze layers gradually
for i in range(2):
    layer_i = list(model.children())[i]
    for param in layer_i.parameters():
        param.requires_grad = True
    lr = (i+1) * 1e-5
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    print("Unfreezing layer", i + 1)

    # Fine-tune
    true_labels, predicted_labels = [], []
    for epoch in range(5):
        model.train()  # Set the model to training mode
        tqdm_val_dl = tqdm(val_dl, desc=f'Epoch {epoch + 1}/{5}')

        for x, y in tqdm_val_dl:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)

            loss.backward()
            optimizer.step()

            true_labels.extend(y.cpu().numpy())
            predicted_labels.extend(torch.argmax(pred, dim=1).cpu().numpy())
            accuracy_val = accuracy_score(true_labels, predicted_labels)

            tqdm_val_dl.set_postfix({'Loss': loss.item(), 'Accuracy': accuracy_val})
    

        tqdm_val_dl.close()


    # Save fine-tuned model
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.item(),
        'accuracy': accuracy_val
    }, PATH)


Loaded pretrained weights for efficientnet-b0
Unfreezing layer 1


Epoch 5/5: 100%|████████████████████████████████████████| 875/875 [05:31<00:00,  2.64it/s, Loss=0.0213, Accuracy=0.952]


Unfreezing layer 2


Epoch 5/5: 100%|████████████████████████████████████████| 875/875 [05:56<00:00,  2.45it/s, Loss=0.0198, Accuracy=0.974]
